# Transfer HTTP (Pull)

With this notebook you can send request to an offered service.

## Preperation

Here some values are initialized that will be necessary for the other steps. Unlike in the HTTP Push, here you will not be able to use a preexisting regular agreement. Instead the Negotiation and Transfer will be handled via the EDR (Endpoint Data Reference) endpoints. For more information, see [TractusX-EDC Walkthrough](https://github.com/eclipse-tractusx/tractusx-edc/blob/0.9.0/docs/usage/management-api-walkthrough/07_edrs.md).


### Set up

Import the requests module and assing the base url of the Dataspace as a variable.

In [ ]:
from pprint import pprint
import requests

base_url = "https://vision-x-api.base-x-ecosystem.org"

### Fill in Values

Fill in the values for the variables below.

In [ ]:
# Your JWT recevied from Keycloak via vision-x-auth.base-x-ecosystem.org
token = "ey..."
token_header = {"Authorization": f"Bearer {token}"}
# The name of your Connector
connector_name = "my-connector"
# The BPN of the Connector providing the offer
provider_id = "BPNL123456789ABC" 
# The DSP address of the Connector providing the offer
originator = "https://vision-x-api.base-x-ecosystem.org/connectors/alice/cp/protocol"
# The policy of the offer
policy = {
    "@id": "some-uuid:some-uuid:some-uuid",
    "@type": "odrl:Offer",
    "odrl:permission": [],
    "odrl:prohibition": [],
    "odrl:obligation": []
}
# The ID of the Asset in the offer
offered_asset_id = "some-random-uuid"

## Initiate EDR Negotiation

Here you start an EDR Negotiation for the Offer chosen in the previous step. This performs the usual Negotiation and Tranfer in such a way that you can later retrieve access tokens. If the given conditions are satisfied the EDR Negotiation will succeed and an Agreement will be created.

Starts the EDR Negotiation using the values got from the previous step.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/edrs"
payload = {
	"@context": {
		"odrl": "http://www.w3.org/ns/odrl/2/"
	},
	"counterPartyAddress": originator,
	"protocol": "dataspace-protocol-http",
	"policy": policy | {"odrl:assigner": {"@id": provider_id}, "odrl:target": {"@id": offered_asset_id}}
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()

negotiation_id = response.json()["@id"]
print(f"Started EDR Negotiation with ID: {negotiation_id}")

As mentioned the EDR starts a Negotiation and a Transfer. Information about the Negotiation can be easily accessed by the below request since we know its ID from the previous step. However, for the transfer we do not know the ID so it is not as easy to get information on it at this point.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/contractnegotiations/{negotiation_id}"

response = requests.get(url, headers=token_header)
response.raise_for_status()
print(f"Negotiation data:\n")
pprint(response.json())

## Get EDR

Here you will get the access token.

Gets all EDRs for the chosen asset.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/edrs/request"
payload = {
  "@context": {},
  "@type": "QuerySpec",
  "filterExpression": [
        {
            "operandLeft": "assetId",
            "operator": "=",
            "operandRight": offered_asset_id
        }
        
    ]
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()

print(f"EDRs data:\n")
pprint(response.json())

transfer_id = response.json()[0]["transferProcessId"]

Uses the Transfer Id from the previous step to get the acccess token.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/edrs/{transfer_id}/dataaddress"

response = requests.get(url, headers=token_header)
response.raise_for_status()

print(f"EDR data:\n")
pprint(response.json())

access_token = response.json()["authorization"]
endpoint = response.json()["endpoint"]

# Access the Service

Now you can use the access token to make requests to the offered service. Your request will go to the provider's Connector which will confirm the existence and validity of the associated Agreeement and then reroute your request to the service. Finally the Connector will send back the response of the service.

In [ ]:
headers = {"Authorization": access_token}

payload = {
    "test": "This is a test."
}

response = requests.post(endpoint, headers=headers, json=payload)

print("Response:")
pprint(response.json())

If everything was successful you should now be able to see the response from the service.